<a href="https://colab.research.google.com/github/ReachetatiVe/CompAlgebra2181Meschaninov/blob/main/5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "Untitled6.ipynb",
      "provenance": [],
      "authorship_tag": "ABX9TyMsT94+dETeetTA+jAYNAiU",
      "include_colab_link": true
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "view-in-github",
        "colab_type": "text"
      },
      "source": [
        "<a href=\"https://colab.research.google.com/github/Enneigardik/Ex/blob/main/Untitled6.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "smjBI4oC2CFM"
      },
      "source": [
        "from IPython.display import display, Math\n",
        "import matplotlib\n",
        "import matplotlib.pyplot as plt"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "rqaficx91Nux"
      },
      "source": [
        "def check(x):\n",
        "  if isinstance(x, Expression):\n",
        "    return x\n",
        "  return Expression(x)\n",
        "\n",
        "class Expression:\n",
        "  def __init__(self, f):\n",
        "    self.f = f\n",
        "  def __add__(self, x):\n",
        "    x = check(x)\n",
        "    return Expression(add(self.f, x.f))\n",
        "  def __sub__(self, x):\n",
        "    x = check(x)\n",
        "    return Expression(sub(self.f, x.f))\n",
        "  def __mul__(self, x):\n",
        "    x = check(x)\n",
        "    return Expression(mul(self.f, x.f))\n",
        "  def __truediv__(self, x):\n",
        "    x = check(x)\n",
        "    return Expression(div(self.f, x.f))\n",
        "  def __pow__(self, x):\n",
        "    x = check(x)\n",
        "    return Expression(pow(self.f, x.f))\n",
        "  def __radd__(self, x):\n",
        "    x = check(x)\n",
        "    return Expression(add(x.f, self.f))\n",
        "  def __rsub__(self, x):\n",
        "    x = check(x)\n",
        "    return Expression(sub(x.f, self.f))\n",
        "  def __rmul__(self, x):\n",
        "    x = check(x)\n",
        "    return Expression(mul(x.f, self.f))\n",
        "  def __rtruediv__(self, x):\n",
        "    x = check(x)\n",
        "    return Expression(div(x.f, self.f))\n",
        "  def __rpow__(self, x):\n",
        "    x = check(x)\n",
        "    return Expression(pow(x.f, self.f))\n",
        "  def __neg__(self):\n",
        "    return Expression(neg(self.f))\n",
        "  def subs(self, var, val):\n",
        "    return Expression(substitute(self.f, var, val))\n",
        "  def evalf(self):\n",
        "    return evaluate(self.f)\n",
        "  def __str__(self):\n",
        "    return expr2latex(self.f)\n",
        "  def show(self):\n",
        "    display(Math(expr2latex(self.f)))\n",
        "  def diff(self, var):\n",
        "    var = check(var).f\n",
        "    return Expression(differentiate(self.f, var))\n",
        "  def simp(self):\n",
        "    return Expression(simplify(self.f))\n",
        "  def expn(self):\n",
        "    return Expression(expand(self.f))\n",
        "  "
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "zwF9OCEU12WJ"
      },
      "source": [
        "def add(x, y):\n",
        "  return ['+', x, y]\n",
        "\n",
        "def mul(x, y):\n",
        "  return ['*', x, y]\n",
        "\n",
        "def div(x, y):\n",
        "  return [\"/\", x, y]\n",
        "\n",
        "def sub(x, y):\n",
        "  return [\"-\", x, y]\n",
        "\n",
        "def pow(x, y):\n",
        "  return [\"^\", x, y]\n",
        "\n",
        "def neg(x):\n",
        "  return [\"--\", x]"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "atEB1lQJ16_S"
      },
      "source": [
        "def fsqrt(x):\n",
        "  return ['sqrt', x]\n",
        "\n",
        "def fsin(x):\n",
        "  return ['sin', x]\n",
        "\n",
        "def fcos(x):\n",
        "  return ['cos', x]\n",
        "\n",
        "def fln(x):\n",
        "  return ['ln', x]"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "poqAG31M3Fdl"
      },
      "source": [
        "def sqrt(x):\n",
        "  x = check(x)\n",
        "  t = x.f\n",
        "  r = fsqrt(t)\n",
        "  return Expression(r)\n",
        "\n",
        "def sin(x):\n",
        "  return Expression(fsin(check(x).f))\n",
        "\n",
        "def cos(x):\n",
        "  return Expression(fcos(check(x).f))\n",
        "\n",
        "def ln(x):\n",
        "  return Expression(fln(check(x).f))"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "-Q4ucQ-q3Jq0"
      },
      "source": [
        "def substitute(f, var, val):\n",
        "  val = check(val).f\n",
        "  if f == var:\n",
        "    return val\n",
        "  elif isinstance(f, list):\n",
        "    y = [f[0]]\n",
        "    for i in range(1, len(f)):\n",
        "      t = substitute(f[i], var, val)\n",
        "      y.append(t)\n",
        "    return y\n",
        "  else:\n",
        "    return f"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "YGV210qg3Ndn"
      },
      "source": [
        "import math\n",
        "\n",
        "def evaluate(f):\n",
        "  if isinstance(f, (int, float)):\n",
        "    return f\n",
        "  elif isinstance(f, list):\n",
        "    if f[0] == \"+\": \n",
        "      return evaluate(f[1]) + evaluate(f[2])\n",
        "    elif f[0] == \"*\":\n",
        "      return evaluate(f[1]) * evaluate(f[2])\n",
        "    elif f[0] == \"sqrt\":\n",
        "      return math.sqrt(evaluate(f[1]))\n",
        "    elif f[0] == \"sin\":\n",
        "      return math.sin(evaluate(f[1]))\n",
        "    elif f[0] == \"cos\":\n",
        "      return math.cos(evaluate(f[1]))\n",
        "  return None\n",
        "  "
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "EZMv2HgY3Td5"
      },
      "source": [
        "def symbols(vars):\n",
        "  return map(Expression, vars.split())\n",
        "\n",
        "x, y = symbols(\"x y\")  "
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "RhREL2w63bwm"
      },
      "source": [
        "def simplify(f):\n",
        "  if isinstance(f, list):\n",
        "    r = [f[0]]\n",
        "    for x in f[1:]:\n",
        "      r.append(simplify(x))\n",
        "    if r[0] == \"+\":\n",
        "      if isinstance(r[1], int) and isinstance(r[2], int):\n",
        "        return r[1] + r[2] \n",
        "      if r[1] == 0: \n",
        "        return r[2]\n",
        "      if r[2] == 0:\n",
        "        return r[1]\n",
        "      if r[1] == r[2]:\n",
        "        return mul(2, r[1])\n",
        "      if isinstance(r[2], list) and r[2][0] == \"--\": # unary minus\n",
        "        return simplify(sub(r[1], r[2][1]))\n",
        "    if r[0] == \"-\":\n",
        "      if isinstance(r[1], int) and isinstance(r[2], int):\n",
        "        return r[1] - r[2]\n",
        "      if r[1] == 0: \n",
        "        return neg(r[2])\n",
        "      if r[2] == 0:\n",
        "        return r[1]\n",
        "      if r[1] == r[2]:\n",
        "        return 0\n",
        "      if isinstance(r[2], list) and r[2][0] == \"--\": # unary minus\n",
        "        return simplify(add(r[1], r[2][1]))\n",
        "    if r[0] == \"*\":\n",
        "      if isinstance(r[1], int) and isinstance(r[2], int):\n",
        "        return r[1] * r[2] \n",
        "      if r[1] == 0 or r[2] == 0: \n",
        "        return 0\n",
        "      if r[1] == 1:\n",
        "        return r[2]\n",
        "      if r[2] == 1:\n",
        "        return r[1]\n",
        "      if r[1] == r[2]:\n",
        "        return pow(r[1], 2)\n",
        "    if r[0] == \"^\":\n",
        "      if isinstance(r[1], int) and isinstance(r[2], int) and r[2] > 0:\n",
        "        return r[1] ** r[2] \n",
        "      if r[2] == 1: \n",
        "        return r[1]\n",
        "    if r[0] == \"sqrt\":\n",
        "      if isinstance(r[1], int):\n",
        "        a = int(math.sqrt(r[1]))\n",
        "        if a * a == r[1]:\n",
        "          return a\n",
        "        return r \n",
        "    return r\n",
        "  return f "
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "hAeC97k96jPW"
      },
      "source": [
        "def expr2latex(x):\n",
        "  if isinstance(x, (int, float)):\n",
        "    return str(x)\n",
        "  if isinstance(x, str):\n",
        "    return x\n",
        "  if isinstance(x, list):\n",
        "    if x[0] == \"+\":\n",
        "      return expr2latex(x[1]) + \"+\" + expr2latex(x[2])\n",
        "    if x[0] == \"-\":\n",
        "      return expr2latex(x[1]) + \"-\" + expr2latex(x[2])\n",
        "    if x[0] == \"*\":\n",
        "      left = expr2latex(x[1])\n",
        "      if isinstance(x[1], list) and x[1][0] == \"+\":\n",
        "        left = \"\\\\left(\" + left + \"\\\\right)\"\n",
        "      right = expr2latex(x[2])\n",
        "      if isinstance(x[2], list) and x[2][0] == \"+\":\n",
        "        right = \"\\\\left(\" + right + \"\\\\right)\"\n",
        "      return left + \"\\\\cdot \" + right\n",
        "    if x[0] == \"/\":\n",
        "      return \"\\\\dfrac{\" + expr2latex(x[1]) + \"}{\" + expr2latex(x[2]) + \"}\"\n",
        "    if x[0] == \"^\":\n",
        "      left = expr2latex(x[1])\n",
        "      if isinstance(x[1], list):\n",
        "        left = \"\\\\left(\" + left + \"\\\\right)\"\n",
        "      right = expr2latex(x[2])\n",
        "      return \"{\" + left + \"}^{\" + right + \"}\"\n",
        "    if x[0] == \"--\":\n",
        "      a = expr2latex(x[1])\n",
        "      if isinstance(x[1], list):\n",
        "        a = \"\\\\left(\" + a + \"\\\\right)\"\n",
        "      return \"-{\" + a + \"}\"\n",
        "    if x[0] == \"sqrt\":\n",
        "      a = expr2latex(x[1])\n",
        "      return \"\\\\sqrt{\" + a + \"}\"\n",
        "    if x[0] == \"sin\":\n",
        "      a = expr2latex(x[1])\n",
        "      if isinstance(x[1], list):\n",
        "        a = \"\\\\left(\" + a + \"\\\\right)\"\n",
        "      return \"\\\\sin{\" + a + \"}\"\n",
        "    if x[0] == \"cos\":\n",
        "      a = expr2latex(x[1])\n",
        "      if isinstance(x[1], list):\n",
        "        a = \"\\\\left(\" + a + \"\\\\right)\"\n",
        "      return \"\\\\cos{\" + a + \"}\"\n",
        "    if x[0] == \"ln\":\n",
        "      a = expr2latex(x[1])\n",
        "      if isinstance(x[1], list):\n",
        "        a = \"\\\\left(\" + a + \"\\\\right)\"\n",
        "      return \"\\\\ln{\" + a + \"}\"\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "knm7PrBR73oE"
      },
      "source": [
        "def expand(f):\n",
        "  if isinstance(f, list):\n",
        "    r = [f[0]]\n",
        "    for x in f[1:]:\n",
        "      r.append(expand(x))\n",
        "    if r[0] == \"*\":\n",
        "      if isinstance(r[1], list) and r[1][0] == \"+\":\n",
        "        a = expand(mul(r[1][1], r[2]))\n",
        "        b = expand(mul(r[1][2], r[2]))\n",
        "        return add(a, b)\n",
        "      if isinstance(r[2], list) and r[2][0] == \"+\":\n",
        "        a = expand(mul(r[1], r[2][1]))\n",
        "        b = expand(mul(r[1], r[2][2]))\n",
        "        return add(a, b)\n",
        "    if r[0] == \"^\":\n",
        "      if isinstance(r[1], list) and r[1][0] == \"+\" and r[2] == 2:\n",
        "        a = pow(r[1][1], 2)\n",
        "        b = mul(2, mul(r[1][1], r[1][2]))\n",
        "        c = pow(r[1][2], 2)\n",
        "        return add(a, add(b, c))\n",
        "    return r\n",
        "  return f\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 76
        },
        "id": "PIJTfO-x6b9D",
        "outputId": "c1256a49-cc89-4c86-b048-44ec975ed86f"
      },
      "source": [
        "z = x--x\n",
        "print(z.f)\n",
        "z.show()\n",
        "z.simp().show()"
      ],
      "execution_count": 28,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "['-', 'x', ['--', 'x']]\n"
          ],
          "name": "stdout"
        },
        {
          "output_type": "display_data",
          "data": {
            "text/latex": "$$x--{x}$$",
            "text/plain": [
              "<IPython.core.display.Math object>"
            ]
          },
          "metadata": {
            "tags": []
          }
        },
        {
          "output_type": "display_data",
          "data": {
            "text/latex": "$$2\\cdot x$$",
            "text/plain": [
              "<IPython.core.display.Math object>"
            ]
          },
          "metadata": {
            "tags": []
          }
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 94
        },
        "id": "rFPvAf6g6sBo",
        "outputId": "472ad3f0-969a-49e2-9fa8-ee68b59b7659"
      },
      "source": [
        "z = sqrt(cos(x))\n",
        "z.simp().show()\n",
        "print(z.f)\n",
        "z.show()\n",
        "z.simp().show()"
      ],
      "execution_count": 35,
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/latex": "$$\\sqrt{\\cos{x}}$$",
            "text/plain": [
              "<IPython.core.display.Math object>"
            ]
          },
          "metadata": {
            "tags": []
          }
        },
        {
          "output_type": "stream",
          "text": [
            "['sqrt', ['cos', 'x']]\n"
          ],
          "name": "stdout"
        },
        {
          "output_type": "display_data",
          "data": {
            "text/latex": "$$\\sqrt{\\cos{x}}$$",
            "text/plain": [
              "<IPython.core.display.Math object>"
            ]
          },
          "metadata": {
            "tags": []
          }
        },
        {
          "output_type": "display_data",
          "data": {
            "text/latex": "$$\\sqrt{\\cos{x}}$$",
            "text/plain": [
              "<IPython.core.display.Math object>"
            ]
          },
          "metadata": {
            "tags": []
          }
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "C67FbpFS6vKR"
      },
      "source": [
        ""
      ],
      "execution_count": null,
      "outputs": []
    }
  ]
}